# Grand Challenge API Client: interact with grand-challenge.org via python


<img src="pipeline.png" />

GC-API provides a handy functionality to interract with grand-challenge.org via python.

Its purpose and broader functionality is discussed in details in [this](https://grand-challenge.org/blogs/grand-challenge-api-client/) blogpost. In order to use gc-api you will also need to obtain a personal API token. The blog above describe what it is and where to find it. 


Lets start from installing the GC-API package with pip:

In [ ]:
pip install gcapi

In this tutorial we will in details go over following steps:

1. [Uploading files to a Reader study on grand-challenge.org](#section_1)
2. [Downloading files from an Archive on grand-challenge.org](#section_2)
3. [Uploading files to an Algorithm on grand-challenge.org for subsequent inference and downloading the results](#section_3)


Lets start from importing nessecary packages

In [9]:
import os
from gcapi import Client

In order to use gc-api you need to obtain your personal token. 

In [13]:
token='my-personal-api-token-to-grand-challange.org'

 <a id=’section_1’></a>
## 1. Uploading files to a [Reader study](https://grand-challenge.org/reader-studies/)

If you are working on a Reader study with grand-challenge.org, you most likely need to upload the cases to your study directory on the platform. Such operation can be easily performed via GC-API. 

You would probably first want to check whether you have been granted permission to create/work with a particular reader study. 

For this particular example, the data on your machine should be structured as follows:

    |--> main_directory 

          |--> patient1_folder --> file_for_a_single_series 

          |--> patient2_folder --> file_for_a_single_series 
          
          |--> patient3_folder --> file_for_a_single_series 
          
                                 ...
          
          |--> patientN_folder --> file_for_a single_series 

Aditionally, you will need to know the "slug" of the particular reader study you will work with. 

For example:

        If the path to your reader study on grand-challenge.org is:
        "https://grand-challenge.org/reader-studies/corads-score-exam/"

        The study slug would be "corads-score-exam"


Lets now list the files in the upload directory to check whether they can be identified:

In [4]:
#Specify the directory path which you want to upload from
upload_from_dir=r'path\to\folder\on\your\machine\with\data\to\upload\to\reader\study'
#Specify the "slug" of the study you want to upload your data to
upload_reader_study_slug="my-reader-stydy-slug"


files = {}

#Loop through files in the specifid directory and add thir names to the list
for root, directories, filenames in os.walk(upload_from_dir):
    for direc in directories:
        temp = []
        for file in os.listdir(os.path.join(root,direc)):
            temp.append(os.path.join(root,direc,file))
        files[direc] = temp

print("Found", len(files.keys()), "cases for upload")



Found 0 cases for upload


Uploading cases

In [6]:
sessions = []

c = Client(token=token)

for i, key in enumerate(files.keys(), 1):
    temp_session = c.upload_cases(files=files[key], reader_study=upload_reader_study_slug)
    sessions.append(temp_session)
    print(files[key])
    print("Uploaded session", i, "of", len(files.keys()))


 <a id=’section_2’></a>
## 2. Downloading files from an [Archive](https://grand-challenge.org/archives/) on grand-challenge.org

In this part we will try downloading cases from grand-challenge.org archive. 

Aditionally, you will need to know the "slug" of the particular archive you will work with. 

For example:

        If the path to your archive on grand-challenge.org is:
        "https://grand-challenge.org/archives/luna16/"

        The archive slug would be "luna16"

In particular we will try to download
chest CT scans of Covid19 patients from the "coronacases.org" dataset of 10 cases.

In [7]:
c = Client(token=token)

#name of the archive arguments
archive_slug = "coronacases.org"
#save path on your machine
output_archive_dir = 'path\to\where\to\save\the\data\to\your\machine'

archives = c(url="https://grand-challenge.org/api/v1/archives/")["results"]

corona_archive = None
for archive in archives:
    if archive["name"] == archive_slug:
        corona_archive = archive
        break
if corona_archive is None:
    raise Exception("archive not found on GC")



In [8]:
# Create image mapping, from image URL to original image mha name
images_mapping = {}
for image_url in corona_archive["images"]:
    # get name of mha image from GC API
    image = c(url=image_url)
    images_mapping[image["files"][0]["file"]] = image["name"]

print("Downloading {0} images..".format(len(images_mapping)))

counter = 0
for file1, name1 in images_mapping.items():
    response = c.get(file1)
    response.raise_for_status()
    with open(os.path.join(output_dir, name1), 'wb') as f1:
        f1.write(response.content)
    counter += 1
    print(counter)



1
2
3
4
5
6
7
8
9
10


 <a id=’section_3’></a>
## 3. Uploading files to an [Algorithm](https://grand-challenge.org/algorithms/) on grand-challenge.org for subsequent inference and downloading the results

In this section we will take a look at how to upload study cases to an Algorithm available on grand-challenge.org and obtain its prediction. 

You will, most likely, first need to request an access to use the desired Algorithm. This can be done by navigating to an algorith directory in grand-challenge.org website and presing "Request access" button.

In this section, we will try to apply a [Pulmonary lobe segmentation algorithm]() from grand-challenge.org to the 10 cases from Covid19 patients downloaded in [part 2](#section_2).

Aditionally, you will need to know the "slug" of the particular algorithm you will work with. 

For example:

        If the path to your algorithm on grand-challenge.org is:
        "https://grand-challenge.org/algorithms/fluid-segmentation-in-retinal-oct/"

        The algorithm slug would be "fluid-segmentation-in-retinal-oct"



Uploading images to an algorithm works in the same manner as when you would upload images to a reader study. However, downloading the results that the algorithm creates is more challenging. The uploaded image gets an universally unique identifier (UUID) which is different from the UUID that gets assigned to the results of the algorithm. For this reason a script was created that uploads the images, tracks the status of the algorithm and directly downloads the results once it is finished. This file can be found in this same repository. Example code on how to use this is shown below.


In [ ]:
from GC_algorithm_API import GrandChallengeAlgorithm


client = GrandChallengeAlgorithm(api_token=token, algorithm="pulmonary-lobe-segmentation")
client.run(input_files=[r'path\to\input\cases\on\your\local\machine'], output_file=r'path\to\output\directory\on\your\local\machine')


